In [1]:
%autoawait

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
import time
import asyncio
import flask
from flask import request

In [3]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://data-agent:8021"

In [4]:
    # Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

In [5]:
def data_messages_hook(payload):
    connection_id = payload["connection_id"]
    asyncio.get_event_loop().create_task(agent_controller.connections.send_message(connection_id, "THIS IS MY RESPONSE - DATA AGENT"))
    print("Handle data messages ", payload, connection_id)


data_message_listener = {
    "handler": data_messages_hook,
    "topic": "basicmessages"
}

In [6]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

<Task pending coro=<AriesAgentController.listen_webhooks() running at /opt/conda/lib/python3.7/site-packages/aries_basic_controller/aries_controller.py:42>>

In [7]:
agent_controller.register_listeners([data_message_listener], defaults=True)

In [8]:
# Create Invitation
invite = await agent_controller.connections.create_invitation(alias="Will")
print(invite)

{'connection_id': '366aa320-3e6b-435c-8415-cdd8b643617a', 'invitation': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '1ddbe7f4-89e3-435f-bb50-946b079b94af', 'label': 'Bob', 'serviceEndpoint': 'http://172.17.0.1:8020', 'recipientKeys': ['GMFbr3hc2Phj9D9YW4JWTaAd3n1itNUxaLjHgVGPBYu5']}, 'invitation_url': 'http://172.17.0.1:8020?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiMWRkYmU3ZjQtODllMy00MzVmLWJiNTAtOTQ2YjA3OWI5NGFmIiwgImxhYmVsIjogIkJvYiIsICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovLzE3Mi4xNy4wLjE6ODAyMCIsICJyZWNpcGllbnRLZXlzIjogWyJHTUZicjNoYzJQaGo5RDlZVzRKV1RhQWQzbjFpdE5VeGFMakhnVkdQQll1NSJdfQ=='}


In [9]:
# Print out accepted Invite and Researcher ID
print("Data Owner ID", invite["connection_id"])
data_connection_id = invite["connection_id"]

Data Owner ID 366aa320-3e6b-435c-8415-cdd8b643617a


In [10]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(data_connection_id)
print("ACCEPT REQUEST")
print(connection)

True
ACCEPT REQUEST
{'invitation_mode': 'once', 'state': 'response', 'initiator': 'self', 'their_did': 'GCWXyezyt8WMjtVBcp3qoJ', 'invitation_key': 'GMFbr3hc2Phj9D9YW4JWTaAd3n1itNUxaLjHgVGPBYu5', 'routing_state': 'none', 'accept': 'manual', 'my_did': 'StD1yXEWmmqSyKPZPdRrAQ', 'their_label': 'Alice', 'created_at': '2020-06-25 15:36:04.467935Z', 'connection_id': '366aa320-3e6b-435c-8415-cdd8b643617a', 'updated_at': '2020-06-25 15:36:45.398857Z'}


In [11]:
trust_ping = await agent_controller.connections.trust_ping(data_connection_id, "hello")
print(trust_ping)

{'thread_id': 'cfebdcaa-baba-4353-87ad-6f1122c19e61'}


In [12]:
connection = await agent_controller.connections.get_connection(data_connection_id)
print(connection)

{'invitation_mode': 'once', 'state': 'active', 'initiator': 'self', 'their_did': 'GCWXyezyt8WMjtVBcp3qoJ', 'invitation_key': 'GMFbr3hc2Phj9D9YW4JWTaAd3n1itNUxaLjHgVGPBYu5', 'routing_state': 'none', 'accept': 'manual', 'my_did': 'StD1yXEWmmqSyKPZPdRrAQ', 'their_label': 'Alice', 'created_at': '2020-06-25 15:36:04.467935Z', 'connection_id': '366aa320-3e6b-435c-8415-cdd8b643617a', 'updated_at': '2020-06-25 15:36:52.807121Z'}


In [13]:
message = await agent_controller.connections.send_message(data_connection_id,"hello from data owner world! WOOPWOOP")
print("BASIC MESSAGE - DATA -> RESEARCH")
print(message)

Handle data messages  {'connection_id': '366aa320-3e6b-435c-8415-cdd8b643617a', 'message_id': 'a1f6d53a-bd3f-40c8-8253-5c1a607948d0', 'content': 'hello from researcher world!', 'state': 'received'} 366aa320-3e6b-435c-8415-cdd8b643617a
BASIC MESSAGE - DATA -> RESEARCH
{}


In [15]:
app = Flask(__name__)

@app.route('/gridhook', methods=['POST'])
def demo():
    if request.method == 'POST':
        content = request.json
        message = await agent_controller.connections.send_message(data_connection_id, content)

SyntaxError: 'await' outside async function (<ipython-input-15-1e06e3d35104>, line 10)